<a href="https://colab.research.google.com/github/aryantyagi0/Train_Video_Processing/blob/main/Copy_of_Untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install -q moviepy



import os, shutil, glob, math, cv2, numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from tqdm import tqdm


In [ ]:

VIDEO_PATH = "/content/DHN_side_view1.mp4"
OUTPUT_ROOT = "Processed_Video"
TRAIN_PREFIX = "DHN_wagon"
SAMPLE_SEC = 0.5
MERGE_SEC = 1.0
MIN_SEG_SEC = 0.6
SAMPLE_FRAME_EVERY = 2
EDGE_OPEN_THRESHOLD = 0.03

In [ ]:
# Cell 3 — thumbnails + histogram-difference candidate generation
os.makedirs(OUTPUT_ROOT, exist_ok=True)
clip = VideoFileClip(VIDEO_PATH)
fps = clip.fps
duration = clip.duration
print(f"Video duration: {duration:.2f}s, fps: {fps}")

sample_times = np.arange(0, duration, SAMPLE_SEC)
thumbs_dir = os.path.join(OUTPUT_ROOT, "thumbs_for_splits")
shutil.rmtree(thumbs_dir, ignore_errors=True)
os.makedirs(thumbs_dir, exist_ok=True)

print("Saving thumbnails...")
sampled_frames = []
for i,t in enumerate(tqdm(sample_times)):
    frame = clip.get_frame(t)
    frame_bgr = cv2.cvtColor((frame*255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    p = os.path.join(thumbs_dir, f"thumb_{i:04d}_{t:.2f}.jpg")
    cv2.imwrite(p, frame_bgr)
    sampled_frames.append((t,p))

# compute HSV hist per thumbnail and Bhattacharyya distance between consecutive thumbnails
def hist_hsv(img_path):
    img = cv2.imread(img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h = cv2.calcHist([hsv], [0,1], None, [50,60], [0,180,0,256])
    cv2.normalize(h, h)
    return h.flatten()

hists = [hist_hsv(p) for _,p in sampled_frames]
dists = [cv2.compareHist(np.float32(hists[i]), np.float32(hists[i+1]), cv2.HISTCMP_BHATTACHARYYA)
         for i in range(len(hists)-1)]
d = np.array(dists)
mean,std = d.mean(), d.std()
threshold = mean + 1.0*std
candidates = [sample_times[i+1] for i in range(len(d)) if d[i] > threshold]


merged = []
for t in candidates:
    if not merged:
        merged.append(t)
    else:
        if t - merged[-1] < MERGE_SEC:
            merged[-1] = (merged[-1] + t)/2.0
        else:
            merged.append(t)
candidates = merged

print("Auto split candidates (seconds):")
print(candidates)
print("\n--> Thumbnails saved to:", thumbs_dir)
print("Inspect thumbnails (Files panel -> open 'thumbs_for_splits') and verify candidates.")


Video duration: 166.68s, fps: 24.25
Saving thumbnails...


100%|██████████| 334/334 [00:11<00:00, 28.69it/s]


Auto split candidates (seconds):
[np.float64(29.75), np.float64(40.0), np.float64(46.0), np.float64(50.0), np.float64(52.5), np.float64(62.75), np.float64(72.0), np.float64(78.0), np.float64(88.0), np.float64(89.5), np.float64(91.0), np.float64(103.5), np.float64(106.5), np.float64(108.0), np.float64(115.75), np.float64(117.0), np.float64(120.0), np.float64(121.0), np.float64(145.0), np.float64(148.75), np.float64(154.0), np.float64(159.0), np.float64(160.0), np.float64(161.0), np.float64(162.0), np.float64(163.5)]

--> Thumbnails saved to: Processed_Video/thumbs_for_splits
Inspect thumbnails (Files panel -> open 'thumbs_for_splits') and verify candidates.


In [ ]:

split_times = candidates.copy()
segments = []
start = 0.0
for t in split_times:
    segments.append((start,float(t)))
    start = float(t)
segments.append((start, duration))
print("Proposed segments:", len(segments))

valid_segments = []
for idx,(s,e) in enumerate(segments, start=1):
    if e - s < MIN_SEG_SEC:
        continue
    folder = os.path.join(OUTPUT_ROOT, f"{TRAIN_PREFIX}_{len(valid_segments)+1}")
    os.makedirs(folder, exist_ok=True)
    out_path = os.path.join(folder, f"{TRAIN_PREFIX}_{len(valid_segments)+1}.mp4")
    print(f"Saving segment {len(valid_segments)+1}: {s:.2f} -> {e:.2f}")
    clip.subclip(s,e).write_videofile(out_path, codec='libx264', audio=False, verbose=False, logger=None)
    valid_segments.append((s,e,folder,out_path))

print("Saved segments:", len(valid_segments))
print("If counts look wrong: re-run Cell 3 with adjusted thresholds or adjust MERGE_SEC / MIN_SEG_SEC.")


Proposed segments: 27
Saving segment 1: 0.00 -> 29.75
Saving segment 2: 29.75 -> 40.00
Saving segment 3: 40.00 -> 46.00
Saving segment 4: 46.00 -> 50.00
Saving segment 5: 50.00 -> 52.50
Saving segment 6: 52.50 -> 62.75
Saving segment 7: 62.75 -> 72.00
Saving segment 8: 72.00 -> 78.00
Saving segment 9: 78.00 -> 88.00
Saving segment 10: 88.00 -> 89.50
Saving segment 11: 89.50 -> 91.00
Saving segment 12: 91.00 -> 103.50
Saving segment 13: 103.50 -> 106.50
Saving segment 14: 106.50 -> 108.00
Saving segment 15: 108.00 -> 115.75
Saving segment 16: 115.75 -> 117.00
Saving segment 17: 117.00 -> 120.00
Saving segment 18: 120.00 -> 121.00
Saving segment 19: 121.00 -> 145.00
Saving segment 20: 145.00 -> 148.75
Saving segment 21: 148.75 -> 154.00
Saving segment 22: 154.00 -> 159.00
Saving segment 23: 159.00 -> 160.00
Saving segment 24: 160.00 -> 161.00
Saving segment 25: 161.00 -> 162.00
Saving segment 26: 162.00 -> 163.50
Saving segment 27: 163.50 -> 166.68


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Saved segments: 27
If counts look wrong: re-run Cell 3 with adjusted thresholds or adjust MERGE_SEC / MIN_SEG_SEC.


In [ ]:
# Cell 5 — sample frames inside each saved coach clip
for seg_idx, (s,e,folder,video_file) in enumerate(valid_segments, start=1):
    cap = cv2.VideoCapture(video_file)
    saved = 0; frame_no = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        if frame_no % SAMPLE_FRAME_EVERY == 0:
            img_path = os.path.join(folder, f"{TRAIN_PREFIX}_{seg_idx}_{saved+1:03d}.jpg")
            cv2.imwrite(img_path, frame)
            saved += 1
        frame_no += 1
    cap.release()
    print(f"Segment {seg_idx}: saved {saved} frames into {folder}")


Segment 1: saved 361 frames into Processed_Video/DHN_wagon_1
Segment 2: saved 125 frames into Processed_Video/DHN_wagon_2
Segment 3: saved 73 frames into Processed_Video/DHN_wagon_3
Segment 4: saved 49 frames into Processed_Video/DHN_wagon_4
Segment 5: saved 31 frames into Processed_Video/DHN_wagon_5
Segment 6: saved 125 frames into Processed_Video/DHN_wagon_6
Segment 7: saved 113 frames into Processed_Video/DHN_wagon_7
Segment 8: saved 73 frames into Processed_Video/DHN_wagon_8
Segment 9: saved 122 frames into Processed_Video/DHN_wagon_9
Segment 10: saved 19 frames into Processed_Video/DHN_wagon_10
Segment 11: saved 19 frames into Processed_Video/DHN_wagon_11
Segment 12: saved 152 frames into Processed_Video/DHN_wagon_12
Segment 13: saved 37 frames into Processed_Video/DHN_wagon_13
Segment 14: saved 19 frames into Processed_Video/DHN_wagon_14
Segment 15: saved 94 frames into Processed_Video/DHN_wagon_15
Segment 16: saved 16 frames into Processed_Video/DHN_wagon_16
Segment 17: saved 37

In [ ]:


for seg_idx, (s,e,folder,video_file) in enumerate(valid_segments, start=1):
    jpgs = sorted(glob.glob(os.path.join(folder, f"{TRAIN_PREFIX}_{seg_idx}_*.jpg")))
    annotated_frames_dir = os.path.join(folder, "annotated_frames")
    shutil.rmtree(annotated_frames_dir, ignore_errors=True)
    os.makedirs(annotated_frames_dir, exist_ok=True)
    segment_boxes = []
    for i, p in enumerate(jpgs):
        img = cv2.imread(p)

        boxes = detect_doors_in_image(img, EDGE_OPEN_THRESHOLD)
        segment_boxes.append((p, boxes))

        annotated_img = img.copy() # Create a copy to draw on
        for (x,y,ww,hh,label, conf) in boxes: # Added conf to unpack
            cv2.rectangle(annotated_img, (x,y), (x+ww, y+hh), (0,255,0), 2)
            cv2.putText(annotated_img, f"{label}: {conf:.2f}", (x, y-8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2) # Display confidence

        outp = os.path.join(annotated_frames_dir, f"{TRAIN_PREFIX}_{seg_idx}_{i+1:03d}_ann.jpg") # Save annotated image with a distinct name
        cv2.imwrite(outp, annotated_img)

    detected_boxes_per_segment[seg_idx] = segment_boxes # Store detection results per segment
    print(f"Processed and annotated {len(jpgs)} frames for coach {seg_idx} using YOLOv8. Annotated images saved to {annotated_frames_dir}.")

Streaming output truncated to the last 5000 lines.
0: 288x640 2 trains, 112.8ms
Speed: 3.9ms preprocess, 112.8ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 116.4ms
Speed: 5.2ms preprocess, 116.4ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 111.0ms
Speed: 4.0ms preprocess, 111.0ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 121.0ms
Speed: 4.0ms preprocess, 121.0ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 114.3ms
Speed: 3.9ms preprocess, 114.3ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 124.6ms
Speed: 5.4ms preprocess, 124.6ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 121.4ms
Speed: 3.8ms preprocess, 121.4ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 trains, 116.9ms
Speed: 3.8ms preprocess,

In [ ]:

import base64

html_path = os.path.join(OUTPUT_ROOT, "DHN_report.html")

with open(html_path, "w") as f:
    f.write("<html><head><title>Door Detection Report</title></head><body>")
    f.write("<h1>Door Detection Report</h1>")
    f.write("<p>If the door open labels are appearing incorrectly, you can try adjusting the `EDGE_OPEN_THRESHOLD` value in Cell 2 and re-run cells 6 and 7.</p>")
    f.write("<p>Note: This report embeds images directly, which increases file size but allows offline viewing.</p>") # Added note about embedding
    f.write("<table border='1' cellspacing='0' cellpadding='5'>")
    f.write("<tr><th>Segment</th><th>Frame</th><th>Detected Objects</th><th>Image</th></tr>")

    for seg_idx, (s,e,folder,video_file) in enumerate(valid_segments, start=1):
        annotated_frames_dir = os.path.join(folder, "annotated_frames") # Directory where annotated frames are saved
        annotated_jpg_files = sorted(glob.glob(os.path.join(annotated_frames_dir, f"{TRAIN_PREFIX}_{seg_idx}_*_ann.jpg"))) # Look for annotated images


        segment_detection_results = detected_boxes_per_segment.get(seg_idx, [])


        for frame_idx, annotated_img_path in enumerate(annotated_jpg_files):

            if frame_idx < len(segment_detection_results):
                original_img_path, boxes = segment_detection_results[frame_idx]
                detected = ", ".join([f"{label} ({conf:.2f})" for (x,y,w,h,label, conf) in boxes]) if boxes else "None" # Include confidence
            else:
                detected = "Detection results not available"



            with open(annotated_img_path, "rb") as img_file:
                img_data = base64.b64encode(img_file.read()).decode('utf-8')
            img_src = f"data:image/jpeg;base64,{img_data}" # Assuming images are JPEGs

            f.write("<tr>")
            f.write(f"<td>{seg_idx}</td>")
            f.write(f"<td>{frame_idx + 1}</td>")
            f.write(f"<td>{detected}</td>") # Display detected objects with confidence
            f.write(f"<td><img src='{img_src}' width='320'></td>") # Use Data URI for annotated image
            f.write("</tr>")

    f.write("</table></body></html>")

print(f"✅ HTML report saved to {html_path}")

✅ HTML report saved to Processed_Video/DHN_report.html


In [ ]:

zip_base = "DHN_processed_videos"
shutil.make_archive(zip_base, 'zip', OUTPUT_ROOT)
zip_path = zip_base + ".zip"
print("Zipped output at:", zip_path)
print("Files created under:", OUTPUT_ROOT)
print("Coach folders:", sorted([d for d in os.listdir(OUTPUT_ROOT) if d.startswith(TRAIN_PREFIX)][:50]))


Zipped output at: DHN_processed_videos.zip
Files created under: Processed_Video
Coach folders: ['DHN_wagon_1', 'DHN_wagon_10', 'DHN_wagon_11', 'DHN_wagon_12', 'DHN_wagon_13', 'DHN_wagon_14', 'DHN_wagon_15', 'DHN_wagon_16', 'DHN_wagon_17', 'DHN_wagon_18', 'DHN_wagon_19', 'DHN_wagon_2', 'DHN_wagon_20', 'DHN_wagon_21', 'DHN_wagon_22', 'DHN_wagon_23', 'DHN_wagon_24', 'DHN_wagon_25', 'DHN_wagon_26', 'DHN_wagon_27', 'DHN_wagon_3', 'DHN_wagon_4', 'DHN_wagon_5', 'DHN_wagon_6', 'DHN_wagon_7', 'DHN_wagon_8', 'DHN_wagon_9']


In [ ]:


import base64
import shutil

def horizontal_coverage(img):
    """Return start and end columns of non-empty pixels."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    cols = np.where(th.sum(axis=0) > 0)[0]
    if len(cols)==0:
        return None
    return int(cols[0]), int(cols[-1])

report_images = {}

html_path = os.path.join(OUTPUT_ROOT, "DHN_report_minimal.html")
with open(html_path, "w") as f:
    f.write("<html><head><title>Minimal Coverage Door Detection Report</title></head><body>")
    f.write("<h1>Minimal Coverage Door Detection Report</h1>")
    f.write(f"<p>Total coaches detected: {len(valid_segments)}</p>")

    for seg_idx, (s,e,folder,video_file) in enumerate(valid_segments, start=1):
        annotated_frames_dir = os.path.join(folder, "annotated_frames")
        annotated_jpg_files = sorted(glob.glob(os.path.join(annotated_frames_dir, f"{TRAIN_PREFIX}_{seg_idx}_*_ann.jpg")))

        # Compute coverage intervals
        intervals = []
        width = None
        for p in annotated_jpg_files:
            img = cv2.imread(p)
            if width is None:
                width = img.shape[1]
            cov = horizontal_coverage(img)
            if cov is None: continue
            intervals.append((p,cov[0],cov[1]))

        uncovered = set(range(width)) if width else set()
        selected = []

        while uncovered and intervals:
            best = None
            best_new = 0
            for p,x0,x1 in intervals:
                new_cover = len([c for c in list(uncovered) if x0 <= c <= x1])
                if new_cover > best_new:
                    best_new = new_cover
                    best = (p,x0,x1)
            if best is None or best_new==0:
                break
            selected.append(best[0])
            for c in range(best[1], best[2]+1):
                uncovered.discard(c)
            intervals = [iv for iv in intervals if iv[0] != best[0]]

        if not selected and annotated_jpg_files:
            selected = [annotated_jpg_files[0]]

        report_images[seg_idx] = selected


        f.write(f"<h2>Coach {seg_idx}</h2>")
        f.write(f"<p>Clip: {os.path.basename(video_file)} | Duration: {e-s:.2f}s | Selected frames: {len(selected)}</p>")
        f.write("<div style='display:flex; flex-wrap: wrap;'>")
        for imgp in selected:

            with open(imgp, "rb") as img_file:
                img_data = base64.b64encode(img_file.read()).decode('utf-8')
            img_src = f"data:image/jpeg;base64,{img_data}"
            f.write(f"<div style='margin:8px'><img src='{img_src}' width=320></div>")
        f.write("</div><hr>")

    f.write("</body></html>")

print(f"✅ Minimal coverage HTML report saved to {html_path}")

for seg_idx, (s,e,folder,video_file) in enumerate(valid_segments, start=1):
    shutil.copy(html_path, os.path.join(folder, f"{TRAIN_PREFIX}_{seg_idx}_report.html"))

print("✅ HTML report also copied to each coach folder")


✅ Minimal coverage HTML report saved to Processed_Video/DHN_report_minimal.html
✅ HTML report also copied to each coach folder
